In [216]:
import pandas as pd
import json
import numpy as np
import spacy

In [177]:
us_videos_df = pd.read_csv('./data/USvideos.csv')

Sorting the videos by video_id and date for later group by operations.

In [178]:
us_videos_df.sort_values(by=['video_id', 'trending_date'], ascending=True, inplace=True)

In [179]:
us_videos_df.head()

,video_id,trending_date,title,channel_title,category_id,publish_time,tags,views,likes,dislikes,comment_count,thumbnail_link,comments_disabled,ratings_disabled,video_error_or_removed,description
90,-2RVw2_QyxQ,17.14.11,2017 Champions Showdown: Day 3,Saint Louis Chess Club,27,2017-11-12T02:39:01.000Z,"Chess|""Saint Louis""|""Club""",67429,438,23,23,https://i.ytimg.com/vi/-2RVw2_QyxQ/default.jpg,False,False,False,The Saint Louis Chess Club hosts a series of f...
346,-2RVw2_QyxQ,17.15.11,2017 Champions Showdown: Day 3,Saint Louis Chess Club,27,2017-11-12T02:39:01.000Z,"Chess|""Saint Louis""|""Club""",69786,453,26,19,https://i.ytimg.com/vi/-2RVw2_QyxQ/default.jpg,False,False,False,The Saint Louis Chess Club hosts a series of f...
584,-2RVw2_QyxQ,17.16.11,2017 Champions Showdown: Day 3,Saint Louis Chess Club,27,2017-11-12T02:39:01.000Z,"Chess|""Saint Louis""|""Club""",71089,460,27,20,https://i.ytimg.com/vi/-2RVw2_QyxQ/default.jpg,False,False,False,The Saint Louis Chess Club hosts a series of f...
1404,-El8z4uy5Ac,17.21.11,The Best Movie Hell to End Up In - After Hours,Cracked,23,2017-11-20T17:00:00.000Z,"movie hells|""hercules""|""bedazzled""|""all dogs g...",327936,11643,584,1305,https://i.ytimg.com/vi/-El8z4uy5Ac/default.jpg,False,False,False,The gang gets together for a frank discussion ...
1621,-El8z4uy5Ac,17.22.11,The Best Movie Hell to End Up In - After Hours,Cracked,23,2017-11-20T17:00:00.000Z,"movie hells|""hercules""|""bedazzled""|""all dogs g...",522256,14312,1062,1553,https://i.ytimg.com/vi/-El8z4uy5Ac/default.jpg,False,False,False,The gang gets together for a frank discussion ...


Retrieving the last video of every group. The last video would be the latest video (as per trending date).

In [180]:
us_grouped_videos = us_videos_df.groupby(['video_id']).last().reset_index()

In [181]:
us_grouped_videos.head()

,video_id,trending_date,title,channel_title,category_id,publish_time,tags,views,likes,dislikes,comment_count,thumbnail_link,comments_disabled,ratings_disabled,video_error_or_removed,description
0,-2RVw2_QyxQ,17.16.11,2017 Champions Showdown: Day 3,Saint Louis Chess Club,27,2017-11-12T02:39:01.000Z,"Chess|""Saint Louis""|""Club""",71089,460,27,20,https://i.ytimg.com/vi/-2RVw2_QyxQ/default.jpg,False,False,False,The Saint Louis Chess Club hosts a series of f...
1,-El8z4uy5Ac,17.26.11,The Best Movie Hell to End Up In - After Hours,Cracked,23,2017-11-20T17:00:00.000Z,"movie hells|""hercules""|""bedazzled""|""all dogs g...",639982,16519,1243,1739,https://i.ytimg.com/vi/-El8z4uy5Ac/default.jpg,False,False,False,The gang gets together for a frank discussion ...
2,-JVITToppE0,17.17.11,Coach Taggart Monday Presser Ahead of Arizona,GoDucksdotcom,17,2017-11-13T20:41:45.000Z,"Oregon|""Ducks""|""college athletics""|""college fo...",704,10,2,5,https://i.ytimg.com/vi/-JVITToppE0/default.jpg,False,False,False,Coach Taggart addresses the media ahead of the...
3,-OK9i1_YYYc,17.16.11,Animal Adventure Park Giraffe Cam,Animal Adventure Park,15,2017-11-12T00:18:43.000Z,"Giraffe|""Zoo""|""New York""|""giraffe cam""|""Animal...",46260,2286,35,17,https://i.ytimg.com/vi/-OK9i1_YYYc/default.jpg,False,False,False,November 2017 Giraffe Cam Hours\n\nEnjoy your ...
4,-SIp6xSP7ds,17.28.11,Gayle King and Norah O'Donnell respond to Char...,CBS This Morning,25,2017-11-21T12:35:39.000Z,"video|""cbs""|""news""|""CBS This Morning""|""Charlie...",534125,2507,1201,1772,https://i.ytimg.com/vi/-SIp6xSP7ds/default.jpg,False,False,False,CBS This Morning co-hosts Norah O'Donnell and ...


Reading categories from json file

In [182]:
with open('./data/US_category_id.json') as data_file:
    data = json.load(data_file)

In [183]:
categories = []
for item in data['items']:
    category = {}
    category['category_id'] = int(item['id'])
    category['title'] = item['snippet']['title']
    categories.append(category)

In [184]:
categories_df = pd.DataFrame(categories)
categories_df.head()

,category_id,title
0,1,Film & Animation
1,2,Autos & Vehicles
2,10,Music
3,15,Pets & Animals
4,17,Sports


In [185]:
us_final_df = us_grouped_videos.merge(categories_df, on = ['category_id'])
us_final_df.rename(columns={'title_y': 'category', 'title_x': 'video_name'}, inplace=True)

In [186]:
us_final_df.tags[2]

'emergence|"ants"|"intelligence"|"ant"|"sum of its part"|"more is more"|"soul"|"conciousness"|"funny"|"fascinating"|"life"|"human"|"cell"|"animation"|"kurzgesagt"|"in a nutshell"'

In [187]:
def splitTags(tag_list):
    tag_list = tag_list.split('|')
    output = ''
    for tag in tag_list:
        output += tag
    return output

In [188]:
us_final_df['tags'] = us_final_df['tags'].apply(splitTags)

In [189]:
us_final_df.columns

Index(['video_id', 'trending_date', 'video_name', 'channel_title',
       'category_id', 'publish_time', 'tags', 'views', 'likes', 'dislikes',
       'comment_count', 'thumbnail_link', 'comments_disabled',
       'ratings_disabled', 'video_error_or_removed', 'description',
       'category'],
      dtype='object')

In [190]:
#get rid of the punctuations and set all characters to lowercase
import re
RE_PREPROCESS = r'\W+|\d+' #the regular expressions that matches all non-characters

#get rid of punctuation and make everything lowercase
#the code belows works by looping through the array of text
#for a given piece of text we invoke the `re.sub` command where we pass in the regular expression, a space ' ' to
#subsitute all the matching characters with
#we then invoke the `lower()` method on the output of the re.sub command
#to make all the remaining characters
#the cleaned document is then stored in a list
#once this list has been filed it is then stored in a numpy array


In [208]:
i = 0
def process_features(desc):
    try:
        return re.sub(RE_PREPROCESS, ' ', desc)
    except:
        return " "


In [209]:
us_final_df['video_features'] = us_final_df['tags'].astype(str) + us_final_df['video_name'].astype(str) \
                        + us_final_df['channel_title'].astype(str) + us_final_df['description'] + us_final_df['category']

In [210]:
us_final_df['video_features'] = us_final_df['video_features'].apply(process_features)

In [211]:
feature_corpus = us_final_df['video_features'].values

In [227]:
feature_corpus = [comment.lower() for comment in feature_corpus]

In [236]:
parser = spacy.load('en')
processed_feature_corpus = [parser(feature) for feature in feature_corpus]



    Only loading the 'en' tokenizer.



In [238]:
processed_feature_corpus[0:10]

[chess saint louis club   champions showdown day  saint louis chess clubthe saint louis chess club hosts a series of four matches november     today features eight rounds of g   plus the g   games of carlsen ding for the first time in recent history each match features no delay or increment join gms yasser seirawan alejandro ramirez maurice ashley cristian chirila and wgm jen shahade for the move by move n nuschesschamps com n     education,
 john green mental floss koalas marsupial joey pouch chlamydia animals history australia zoo buck mammel bear eons hank green eucalyptus camels lindsay military imperial force home range trees pap mundu   facts about koalas mental_floss list show ep  mental flossa weekly show where knowledge junkies get their fix of trivia tastic information this week john tells you about everyone s favorite marsupial the koala n nsubscribe for new episodes of mental_floss every other wednesday n n nwebsite http www mentalfloss com ntwitter http www twitter com men